In [ ]:
import os

In [ ]:
import sys
print("SYS", sys.version)

In [ ]:
from collections import OrderedDict

In [ ]:
import datetime
from datetime import timedelta

In [ ]:
from dateutil import parser as date_parser
from dateutil import _version as dateutil_version
print("dateutil", dateutil_version.version)

In [ ]:
import pandas as pd
print("pandas", pd.__version__)

In [ ]:
from IPython.display import display
from IPython import __version__ as IPython_version
print("IPython", IPython_version)

In [ ]:
infolder  = '2018-Auris/Logs'

In [ ]:
DB_FOLDER = 'db'

In [ ]:
MIN_TIME_BETWEEN_TRIPS_IN_MINUTES = 5

In [ ]:
months    = ',Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec'.split(',')

In [ ]:
prefix    = infolder.replace('/','_')

In [ ]:
MIN_TIME_BETWEEN_TRIPS_IN_SECONDS = MIN_TIME_BETWEEN_TRIPS_IN_MINUTES * 60

In [ ]:
#https://stackoverflow.com/questions/3463930/how-to-round-the-minute-of-a-datetime-object-python
def roundTime(dt=None, roundTo=1):
   """Round a datetime object to any time laps in seconds
   dt : datetime.datetime object, default now.
   roundTo : Closest number of seconds to round to, default 1 minute.
   Author: Thierry Husson 2012 - Use it as you want but don't blame me.
   """
   if dt == None : dt = datetime.datetime.now()
   seconds  = (dt.replace(tzinfo=None) - dt.min).seconds
   rounding = (seconds+roundTo/2) // roundTo * roundTo
   return dt + datetime.timedelta(0,rounding-seconds,-dt.microsecond)

In [ ]:
def toFloat(val):
    try:
        return float(val)
    except ValueError:
        return None

In [ ]:
def toDate(val):
    try:
        d = date_parser.parse(val)
        #e = d.astimezone(tz=None)
        f = d.replace(tzinfo=None)
        g = roundTime(f,roundTo=1)
        return g
    except ValueError:
        return None

In [ ]:
def guessType(val):
    if val is None:
        return lambda x: None
    
    try:
        val = float(val)
#         print("guessType FLOAT", val)
        return toFloat
    except:
        pass

    try:
        val = date_parser.parse(val)
#         print("guessType DATE", val)
        return toDate
    except:
        pass
    
#     print("guessType STR", val)
    return lambda x: str(x)

In [ ]:
files    = [f for f in os.listdir(infolder) if f.endswith('.csv') and f.startswith('trackLog-') and not os.path.isdir(f) ]

In [ ]:
print( "\n".join( files ) )

In [ ]:
assert len(files) > 0, "NO FILES FOUND"

In [ ]:
orderedFiles = {}

for f in files:
    g          = f.replace('trackLog-','').replace('.csv','')
    date, hour = g.split("_")
    yr, mo, da = date.split('-')
    hr, mi, se = [int(h) for h in hour.split('-')]
    yr, da     = int(yr), int(da)
    mo         = months.index(mo)
    orderedFiles[(yr, mo, da, hr, mi, se)] = os.path.join(infolder, f)

orderedFiles = OrderedDict(sorted(orderedFiles.items()))

#print(orderedFiles)

In [ ]:
data    = OrderedDict()
headers = []
types   = []

for (yr, mo, da, hr, mi, se), f in orderedFiles.items():
    print("{2:02d}/{1:02d}/{0:02d} {3:02d}:{4:02d}:{5:02d} - {6}".format(yr, mo, da, hr, mi, se, f))
    with open(f, 'r') as fhd:
        for ln, line in enumerate(fhd):
            line = line.strip()
#             print(line)
            #if ln == 20: break
            cols = [ l.strip() for l in line.split(',') ]
            
            if cols[-1] == '':
                cols = cols[:-1]
                
            if cols[0] == "GPS Time":
                headers = cols
                types   = [None]*len(headers)
#                 print(headers)
                for h in headers:
                    if h not in data:
                        if len(data.keys()) > 0:
                            data[h] = [None] * len(data[list(data.keys())[0]])
                        else:
                            data[h] = []
            else:
                #assert len(cols) == len(headers), "number of columns for file {} line {} differs {} != {}".format(
                #    f, ln, len(cols), len(headers))
                
                if len(cols) != len(headers):
                    print("number of columns for file {} line {} differs {} != {}".format(
                    f, ln, len(cols), len(headers)))
                    
                    continue

                for h in data:
                    if h in headers:
                        hp  = headers.index(h)
                        val = cols[hp]
                        if val in ['-', '', u'∞']:
                            data[h].append( None )
                        else:
                            if types[hp] is None:
                                types[hp] = guessType(val)
                            val = types[hp](val)
                            data[h].append( val  )
                    else:
                        data[h].append( None )

In [ ]:
#for f, d in data.items():
#    print(u"{:55s} {:12,d}".format(f, len(d)))

In [ ]:
df = pd.DataFrame.from_dict(data)
df.dropna(axis=1, how='all', inplace=True)
# with pd.option_context("display.max_columns",0):
#     display(df)

In [ ]:
cols = list(df)
cols
nunique = df.apply(pd.Series.nunique)
nunique
cols_to_drop = nunique[nunique == 1].index
cols_to_drop
df.drop(cols_to_drop, axis=1, inplace=True)

In [ ]:
with pd.option_context("display.max_columns",0):
    display(df)

In [ ]:
print("\n".join(["{:55} {}".format(k,v) for k,v in sorted(zip(df.dtypes.keys(), df.dtypes.values))]))

In [ ]:
print(df.shape)
df.dropna(subset=['GPS Time'],inplace=True)
df.reset_index(inplace=True, drop=True)
# df.drop(columns=['level_0'], inplace=True)
print(df.shape)
# df

In [ ]:
df["GPS Time DIff"] = df["GPS Time"] - df["GPS Time"].shift()
df.iloc[0,df.columns.get_loc("GPS Time DIff")] = timedelta(seconds=MIN_TIME_BETWEEN_TRIPS_IN_SECONDS*2)
df["GPS Time DIff"]

In [ ]:
df["GPS Time New Trip"] = df["GPS Time DIff"] >= timedelta(seconds=MIN_TIME_BETWEEN_TRIPS_IN_SECONDS)
#df["GPS Time New Trip"]

In [ ]:
df["GPS Time Trip Num"]  = df["GPS Time New Trip"].cumsum()
# df["GPS Time Trip Num"]

In [ ]:
df["GPS Time Trip ID"] = df["GPS Time"]

for trip_num in df["GPS Time Trip Num"].unique():
    trip_vals    = df[["GPS Time","GPS Time Trip Num"]][df["GPS Time Trip Num"] == trip_num]
    num_vals     = len(trip_vals)
    min_time     = trip_vals["GPS Time"].min()
    max_time     = trip_vals["GPS Time"].max()
    del_time     = max_time - min_time
    min_time_str = str(min_time).replace('-','_').replace(':','_').replace(' ','_')
    max_time_str = str(max_time).replace('-','_').replace(':','_').replace(' ','_')
    del_time_str = str(del_time).replace('-','_').replace(':','_').replace(' ','_')
    
    print("trip_num", trip_num, 
          "num_vals", num_vals, 
          "min_time", min_time_str, 
          "max_time", max_time_str, 
          "del_time", del_time_str)
    
    df.iloc[trip_vals.index , df.columns.get_loc("GPS Time Trip ID")] = min_time_str
    
# df["GPS Time Trip ID"]
# df["GPS Time Trip ID"] = df["GPS Time Trip ID"].astype('category')
df["GPS Time Trip ID"].unique(), len(df["GPS Time Trip ID"].unique())

In [ ]:
df[["GPS Time", "GPS Time Trip Num", "GPS Time Trip ID"]][df["GPS Time New Trip"]]

In [ ]:
# df

In [ ]:
print("shape before", df.shape)

df2 = df.drop_duplicates(subset="GPS Time", keep="last")
#         .drop(["GPS Time DIff","GPS Time New Trip"], 1)

print("shape after ", df2.shape)

In [ ]:
df3 = df2.groupby(df2["GPS Time Trip ID"], as_index=True, sort=False, group_keys=True)

In [ ]:
groups = list(df3.groups.keys())

In [ ]:
display(df3.get_group(groups[0]))

In [ ]:
display(df3.get_group(groups[-1]))

In [ ]:
xlsx = os.path.join(DB_FOLDER, "{}_batch_from_{}-to_{}.xlsx".format( prefix, groups[0], groups[-1]))

print('saving to Excel', xlsx)

df2.to_excel(xlsx)

In [ ]:
for gn, group in enumerate(df3.groups):
    g            = df3.get_group(group)
    h            = g.dropna(axis=1, how='all')
    cols         = list(h)
    nunique      = h.apply(pd.Series.nunique)
    cols_to_drop = nunique[nunique == 1].index
    i            = h.drop(cols_to_drop, axis=1)
    
    xlsx = os.path.join(DB_FOLDER, "{}_trackLog-{}.xlsx".format( prefix, group))
    
    shapes = " | ".join(["{} - rows {:5,d} columns {:5,d}".format(n, r, c) for n, (r, c) in (  ('all' , g.shape), 
                                                                                               ('na'  , h.shape), 
                                                                                               ('drop', i.shape))])
    
    print('saving to Excel {:3d}/{:3d} {}\n\t{}'.format(gn+1, len(df3.groups), xlsx, shapes))
    i.to_excel(xlsx)

#     display( g )

In [ ]:
for f in orderedFiles.values():
#     t = f+".done"
#     print( "renaming", f, "to", t )
#     os.rename(f, t)
    if os.path.exists(f):
        print( "compressing", f )
        !gzip -1 $f
    else:
        print( f, "already compressed" )